In [ ]:
from surianalytics.connectors import RESTSciriusConnector

In [ ]:
from surianalytics.widgets.config import update_connector_time_delta

In [ ]:
CONNECTOR = RESTSciriusConnector()

In [ ]:
CONNECTOR.set_page_size(10000)

In [ ]:
update_connector_time_delta(c=CONNECTOR, dump_path="../config.json")

In [ ]:
from IPython.display import JSON

In [ ]:
CLIENTS = (
    CONNECTOR
    .get_eve_unique_values(qfilter="event_type: ssh", 
                           field="ssh.client.software_version", 
                           counts="no")
)

In [ ]:
CLIENTS

In [ ]:
SERVERS = (
    CONNECTOR
    .get_eve_unique_values(qfilter="event_type: ssh", 
                           field="ssh.server.software_version", 
                           counts="no")
)

In [ ]:
SERVERS

In [ ]:
import ipywidgets as widgets

In [ ]:
import pandas as pd

In [ ]:
from ipywidgets.widgets.interaction import display

In [ ]:
def show(limit: int,
         value: str,
         kind: str):
    pd.set_option('display.max_rows', limit)
    pd.set_option('display.min_rows', limit)
    
    df = (
        CONNECTOR
        .get_events_df(qfilter="ssh.{kind}.software_version: {val}".format(kind=kind, val=value))
    )
    
    cols = ["timestamp", "flow_id", "src_ip", "dest_ip", "src_port", "dest_port"]
    
    for col in ["alert.signature", "host"]:
        if col in list(df.columns.values):
            cols = cols + [col]
            
    cols += [c for c in list(df.columns.values) if c.startswith("ssh.")]

    display(df.sort_values(by=["timestamp"])[cols])
    display(df.groupby("ssh.{kind}.software_version".format(kind=kind)).agg({
        "src_ip": ["nunique", "unique"],
        "dest_ip": ["nunique", "unique"],
    }))

In [ ]:
def show_server(limit=widgets.IntSlider(min=10, max=1000),
                server=widgets.Dropdown(options=SERVERS)):
    return show(limit=limit, value=server, kind="server")

In [ ]:
def show_client(limit=widgets.IntSlider(min=10, max=1000),
                client=widgets.Dropdown(options=CLIENTS)):
    return show(limit=limit, value=client, kind="client")

In [ ]:
widgets.interact(show_server)

In [ ]:
widgets.interact(show_client)